In [1]:
import config as conf
import data_preparation as pr
import features_extract as ex

In [2]:
# DATA PREPARATION
train_events = conf.TRAIN_EVENTS
train_submissions = conf.TRAIN_SUBMISSIONS
test_events = conf.TEST_EVENTS
test_submissions = conf.TEST_SUBMISSIONS

In [3]:
train_data =pr.data_preparation(train_events, train_submissions)
test_data = pr.data_preparation(test_events, test_submissions)

In [4]:
X, y = ex.features_extract(train_data)

In [5]:
X

array([[-1.72423514, -0.50183933, -0.63006947, ..., -0.5115427 ,
        -0.56845928, -0.31269911],
       [-1.7241061 , -0.17957779, -0.18654889, ..., -0.26260189,
        -0.31903894, -0.31269911],
       [-1.72397707,  0.14268375,  0.14609154, ..., -0.01366107,
         0.02391402,  0.12771863],
       ...,
       [ 1.73299048,  3.04303763,  2.08649404, ...,  2.47574708,
         2.20634198,  0.45803193],
       [ 1.73337759, -0.17957779, -0.13110882, ..., -0.26260189,
        -0.2878614 , -0.31269911],
       [ 1.73350662, -0.17957779, -0.02022868, ..., -0.13813148,
        -0.22550632, -0.20259468]])